In [3]:
# See page 308 of Deep Learning with Python

import keras
from keras import layers
from keras.preprocessing import image
import numpy as np

import os
import time

latent_dim = 32
height = 32
width = 32
channels = 3

dropout_rate = 0.4

In [4]:
# Define the generator

generator_input = keras.Input(shape=(latent_dim,))

# First, transform the input into a 16x16 128-channels feature map
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# Then, add a convolution layer
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Upsample to 32x32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# Few more conv layers
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Produce a 32x32 1-channel feature map
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [5]:
# Define the discriminator
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

# One dropout layer - important trick!
x = layers.Dropout(0.4)(x)

# Classification layer
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# To stabilize training, we use learning rate decay
# and gradient clipping (by value) in the optimizer.
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

In [6]:
# Freeze the discriminator
discriminator.trainable = False

# Chain the generator and discriminator
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')


In [9]:
# Training using the CIFAR10 data set
(x_train_raw, y_train_raw), (_, _) = keras.datasets.cifar10.load_data()

In [10]:
# Select frog images (class 6)
x_train = x_train_raw[y_train_raw.flatten() == 6]

# Normalize data
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = 'gan-output'


In [14]:
new_weights = None
new_gen_image = None
new_rand_vec = None

start_time = time.time()

# Start training loop
start = 0
for step in range(iterations):
    # Sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Decode them to fake images
    generated_images = generator.predict(random_latent_vectors)

    # Combine them with real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # Assemble labels discriminating real from fake images
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # Add random noise to the labels - important trick!
    labels += 0.05 * np.random.random(labels.shape)

    # Train the discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Assemble labels that say "all real images"
    misleading_targets = np.zeros((batch_size, 1))

    # Train the generator (via the gan model,
    # where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0

    # Occasionally save / plot
    if step % 100 == 0:
        old_weights = new_weights
        new_weights = generator.get_weights()
        
        old_gen_image = new_gen_image
        new_gen_image = generated_images[0]
        
        old_rand_vec = new_rand_vec
        new_rand_vec = random_latent_vectors[0]
        
        if old_weights is not None:
            ave_weight_diffs = [np.average(old_weight - new_weight) for old_weight, new_weight in zip(old_weights, new_weights)]
            print()
            print('Average change is generator weights by layer : {}'.format(ave_weight_diffs))
            print('Average pixel diff for generated image: {}'.format(np.average(old_gen_image - new_gen_image)))
            print('Difference in latent space vector: {}'.format(old_rand_vec - new_rand_vec))
            print()
        
         gan.save_weights('gan.h5')
        # Print metrics
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

        # Save one generated image
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))

        # Save one real image, for comparison
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))
        
        
        current_time = time.time()
        print('Elapsed time: {}'.format(current_time - start_time))

/Users/william.nelson/.pyenv/versions/3.6.1/envs/cv3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss at step 0: 0.69627845
adversarial loss at step 0: 0.7561928

Average change is generator weights by layer : [-5.347755e-05, 0.004382328, -0.00042639833, 0.0012079914, 0.00049717195, 0.00091081247, -6.8319474e-05, 0.000369424, -0.00029494637, -0.0023300964, -0.00061414536, -0.009193701]
Average pixel diff for generated image: 0.08178230375051498
Difference in latent space vector: [ 0.9674479   0.68065484 -2.93921547 -1.20700099  2.11113566  1.2683813
  0.9941654  -1.69400602  0.69560748  0.3116504   0.05081979 -0.02799654
  0.2741191  -1.05501103  1.57229459 -0.12294069  2.95112505  1.02792008
 -0.69870441 -1.77972918 -1.53858262 -0.35138888  0.02046437 -2.48253783
 -1.97561531  1.79274042 -0.52038867 -1.29014653 -0.06812047  2.48066013
  2.04529541 -0.48487529]

discriminator loss at step 100: 0.6902386
adversarial loss at step 100: 0.9056079

Average change is generator weights by layer : [-6.526088e-05, 0.00056622474, 4.422502e-06, 0.0014011005, 3.83831e-05, 0.0010


Average change is generator weights by layer : [3.5165103e-05, 0.00048315356, 7.567555e-06, 0.00055773393, 0.0002697041, 0.00028598006, 6.077902e-05, -4.5419343e-05, 5.83914e-05, -0.0007503469, 0.00017303569, -0.0024711203]
Average pixel diff for generated image: -0.33902668952941895
Difference in latent space vector: [ 1.97790657 -2.83381533 -0.47254675 -1.57140465 -0.75731813  1.09509689
 -0.66853865  0.76083514  1.17458066  0.36776954 -0.49513045  2.22400589
  0.04519342 -1.28557962  0.31234079 -1.95181215 -1.12051228  0.7243248
 -0.38404019 -1.01806562 -0.37408315 -2.26503239  1.11092553  0.96489277
 -3.20599192 -1.69311752 -0.2669928   2.00306359 -2.42815839 -0.21305189
 -0.35755034  0.47795586]

discriminator loss at step 1200: 0.7003213
adversarial loss at step 1200: 0.8071721

Average change is generator weights by layer : [7.1875635e-05, 0.000429005, 8.304324e-05, 0.00086217845, -9.3325536e-05, -2.4123117e-05, 3.1877702e-05, -5.765349e-05, 6.289445e-05, 0.00018099057, 6.23613

KeyboardInterrupt: 